In [1]:
import pandas as pd
import dataframe_image as dfi
import requests
import os
import json
import requests
import warnings
from bs4 import BeautifulSoup
os.makedirs("./Result",exist_ok=True)



# 國內成分證券ETF
https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html

In [2]:


url='https://www.twse.com.tw/rwd/zh/ETF/domestic?response=json&_=1681734086399'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}
req = requests.get(url, headers = headers,verify=False)
ETFdata = req.content.decode('utf8')
ETFdf=pd.DataFrame(json.loads(ETFdata)['data'])
ETFdf=ETFdf.dropna()
ETFdf.columns = ['證券代號', '證券簡稱','']

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



# ETF   持股 Data

In [3]:
# https://www.moneydj.com/ETF/X/Basic/Basic0007A.xdjhtm?etfid=0050.TW
# https://www.moneydj.com/etf/eb/et305001list.djhtm?order=4&c=47
def get_erfBaseData(etfid,rtfdf):
    etfurl='https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid='+etfid+'.TW'
    etfresp = requests.get(etfurl)
    etfresp.encoding = 'utf-8'    #轉換編碼至UTF-8
    print("utl="+etfurl)
    if etfresp.status_code==200 :
        etfsoup = BeautifulSoup(etfresp.text, 'lxml')
        
        #印出表格中的所有文字內容
        for tr in etfsoup.find('table', id="Repeater1").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            row.append(etfid)
            if len(row)>0:
                rtfdf = rtfdf.append([row],ignore_index=True)
        for tr in etfsoup.find('table', id="Repeater2").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            if len(row)>0:
                row.append(etfid)
                rtfdf = rtfdf.append([row],ignore_index=True)

        rtfdf=rtfdf.dropna()
       
        return rtfdf

In [4]:
rtfdf = pd.DataFrame()
for  etfid in ETFdf['證券代號'].tolist():
    rtfdf=get_erfBaseData(etfid,rtfdf)
# 【Output】   
rtfdf.columns = ['股票名稱', '持股(千股)','比例','增減','ETF_Id']
rtfdf.to_csv(r'BaseInfo/ETFBaseData.csv', index = False, encoding='utf-8', errors='ignore')
rtfdf.to_csv(r'Result/ETFBaseDataRead.csv', index = False,encoding='big5', errors='ignore')

utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW


C:\Users\user\AppData\Local\Temp\ipykernel_4680\1022840050.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_4680\1022840050.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)


utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0051.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0052.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0053.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0055.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0056.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0057.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006203.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006204.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006208.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00690.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00692.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00701.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00713.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdj

In [5]:



StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv')
ETFBaseData=pd.read_csv('BaseInfo/ETFBaseData.csv')

In [6]:
# 兩表Join
df=pd.merge(ETFBaseData, StockBaseData, how='left', left_on='股票名稱', right_on='公司簡稱')
# 處理格式問題
df['ETF_Id']='00'+df['ETF_Id'].astype(str)

#處理上市公司產業類別對應不到的狀況
df['上市公司產業類別']=df['上市公司產業類別'].fillna('未分類')
print(df.columns)

Index(['股票名稱', '持股(千股)', '比例', '增減', 'ETF_Id', '公司代號', '公司名稱', '公司簡稱', '產業別',
       '實收資本額', 'Type', '上市公司產業類別', '上櫃公司產業類別', '新編碼'],
      dtype='object')


In [7]:
# 取得 etf 對應持股佔比
df['data']=df.agg(lambda x: f"{x['股票名稱']} ( {x['比例']}%)", axis=1)
df.sort_values('比例', ascending=False)
t=df[['ETF_Id','data']]
t[t['ETF_Id']=='00713']

,ETF_Id,data


In [8]:
keydata=df.groupby(['ETF_Id','上市公司產業類別'])['data'].apply(','.join).reset_index()
keydata

,ETF_Id,上市公司產業類別,data
0,000050,光電業,大立光 ( 0.55%)
1,000050,其他,中租-KY ( 0.44%)
2,000050,其他電子業,鴻海 ( 4.33%)
3,000050,半導體業,"台積電 ( 56.12%),聯發科 ( 5.14%),聯電 ( 1.37%),日月光投控 (..."
4,000050,塑膠工業,"南亞 ( 0.41%),台塑 ( 0.33%)"
...,...,...,...
812,0000980A,半導體業,"台積電 ( 3.82%),聯發科 ( 3.33%),矽創 ( 1.45%)"
813,0000980A,通信網路業,智邦 ( 1.46%)
814,0000980A,金融保險,"富邦金 ( 3.15%),中信金 ( 2.01%)"
815,0000980A,電子零組件業,"台達電 ( 2.66%),台虹 ( 1.43%)"


In [9]:
# 取得產業比例排名 by ETF_Id 上市公司產業類別

# 挑選要用的欄位
colList=['ETF_Id','股票名稱', '上市公司產業類別','持股(千股)','比例']
df_=df[colList]
# groupby
g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()
# 排序 取前三
#g.sort_values('比例', ascending=False).groupby(['ETF_Id']).head(3).sort_values(['ETF_Id', '比例'], ascending=[True, False])
# 排序(這個對未來比較有彈性)
g['row_numbers'] =g.groupby('ETF_Id')['比例'].rank(ascending=False,method='dense')
g[g['ETF_Id']=='00713']

C:\Users\user\AppData\Local\Temp\ipykernel_4680\2400789445.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()


,ETF_Id,上市公司產業類別,比例,row_numbers


In [10]:
#轉成橫的方便看
df1=g[g['row_numbers']==1].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df2=g[g['row_numbers']==2].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df3=g[g['row_numbers']==3].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df_result=df1.merge(df2,on='ETF_Id',suffixes=["","_2"]).merge(df3,on='ETF_Id',suffixes=["","_3"])
df_result = df_result.drop(df_result.filter(regex='row_numbers').columns, axis=1)
df_result

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,000050,半導體業,65.57,"台積電 ( 56.12%),聯發科 ( 5.14%),聯電 ( 1.37%),日月光投控 (...",金融保險,12.15,"富邦金 ( 1.76%),中信金 ( 1.65%),國泰金 ( 1.38%),兆豐金 ( 1...",電腦及週邊設備業,5.65,"廣達 ( 1.63%),華碩 ( 1.06%),緯創 ( 0.71%),光寶科 ( 0.53..."
1,000051,電子零組件業,13.59,"台光電 ( 2.55%),川湖 ( 1.64%),健策 ( 1.56%),嘉澤 ( 1.38...",半導體業,10.73,"矽力*-KY ( 1.99%),京元電子 ( 1.46%),創意 ( 1.37%),祥碩 (...",電腦及週邊設備業,9.90,"技嘉 ( 1.78%),仁寶 ( 1.68%),英業達 ( 1.58%),宏碁 ( 1.51..."
2,000052,半導體業,79.13,"台積電 ( 65.88%),聯發科 ( 6.04%),聯電 ( 1.6%),日月光投控 ( ...",電腦及週邊設備業,8.61,"廣達 ( 1.91%),華碩 ( 1.24%),緯創 ( 0.83%),光寶科 ( 0.62...",其他電子業,5.98,"鴻海 ( 5.1%),可成 ( 0.4%),貿聯-KY ( 0.28%),鴻準 ( 0.2%)"
3,000053,半導體業,65.53,"台積電 ( 52.11%),聯發科 ( 5.5%),聯電 ( 1.35%),日月光投控 ( ...",電腦及週邊設備業,9.10,"廣達 ( 2.05%),華碩 ( 0.94%),緯穎 ( 0.83%),緯創 ( 0.66%...",電子零組件業,7.74,"台達電 ( 2.6%),川湖 ( 0.48%),台光電 ( 0.41%),嘉澤 ( 0.37..."
4,000056,電腦及週邊設備業,22.94,"華碩 ( 3.51%),光寶科 ( 2.57%),和碩 ( 2.57%),廣達 ( 2.41...",半導體業,19.92,"聯電 ( 4.14%),聯詠 ( 3.97%),瑞昱 ( 3.73%),聯發科 ( 3.25...",金融保險,16.94,"中信金 ( 3.6%),華南金 ( 3.21%),上海商銀 ( 2.91%),兆豐金 ( 2..."
5,000057,半導體業,61.04,"台積電 ( 51.36%),聯發科 ( 4.63%),聯電 ( 1.15%),日月光投控 (...",金融保險,11.57,"富邦金 ( 1.56%),中信金 ( 1.48%),國泰金 ( 1.26%),兆豐金 ( 1...",電腦及週邊設備業,6.26,"廣達 ( 1.45%),華碩 ( 0.92%),緯創 ( 0.63%),緯穎 ( 0.48%..."
6,00006203,半導體業,61.20,"台積電 ( 51.29%),聯發科 ( 4.74%),聯電 ( 1.16%),日月光投控 (...",金融保險,11.66,"富邦金 ( 1.58%),中信金 ( 1.49%),國泰金 ( 1.27%),兆豐金 ( 1...",電腦及週邊設備業,6.35,"廣達 ( 1.47%),華碩 ( 0.95%),緯創 ( 0.63%),緯穎 ( 0.5%)..."
7,00006204,半導體業,44.86,"台積電 ( 34.7%),聯發科 ( 4.16%),聯電 ( 1.22%),日月光投控 ( ...",金融保險,13.94,"富邦金 ( 2.18%),國泰金 ( 1.65%),中信金 ( 1.48%),兆豐金 ( 1...",電腦及週邊設備業,7.25,"廣達 ( 1.78%),華碩 ( 0.83%),緯穎 ( 0.64%),緯創 ( 0.57%..."
8,00006208,半導體業,65.56,"台積電 ( 56.13%),聯發科 ( 5.13%),聯電 ( 1.36%),日月光投控 (...",金融保險,12.15,"富邦金 ( 1.76%),中信金 ( 1.65%),國泰金 ( 1.38%),兆豐金 ( 1...",電腦及週邊設備業,5.62,"廣達 ( 1.62%),華碩 ( 1.05%),緯創 ( 0.71%),光寶科 ( 0.53..."
9,0000690,半導體業,59.99,"台積電 ( 31.62%),聯發科 ( 16.1%),聯電 ( 4.18%),聯詠 ( 2....",電子零組件業,11.31,"台達電 ( 5.56%),國巨 ( 1.68%),台光電 ( 1.38%),欣興 ( 0.9...",金融保險,8.27,"富邦金 ( 6.58%),上海商銀 ( 1.69%)"


In [11]:

# export png
df_styled = df_result.style.background_gradient()
df_styled = df_result.style.set_properties(subset=['data','data_2','data_3'], **{'width': '600px'})
dfi.export(df_styled, "Result/mytable.png")

In [12]:
df_styled = (
    df_result.style
    .background_gradient()
    .set_properties(subset=['data', 'data_2', 'data_3'], **{'width': '600px'})
)

dfi.export(df_styled, "Result/mytable.png")